In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
bike_df = pd.read_csv('./train.csv')
bike_df['datetime'] = pd.to_datetime(bike_df['datetime'])
bike_df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [43]:
bike_df['year'] = bike_df['datetime'].dt.year # year는 숫자형으로 두는 경우도 많습니다.
bike_df['month'] = bike_df['datetime'].dt.month
bike_df['dayofweek'] = bike_df['datetime'].dt.dayofweek
bike_df['hour'] = bike_df['datetime'].dt.hour

In [11]:
bike_df['year'] = bike_df['datetime'].dt.year # year는 숫자형으로 두는 경우도 많습니다.
bike_df['month'] = bike_df['datetime'].dt.month
bike_df['dayofweek'] = bike_df['datetime'].dt.dayofweek
bike_df['hour'] = bike_df['datetime'].dt.hour

In [12]:
bike_df =bike_df.drop(['datetime'], axis=1)


In [13]:
X = bike_df.drop(['count', 'casual', 'registered'], axis=1)
y = bike_df['count']

In [14]:
categorical_features_to_encode = ['season', 'holiday', 'workingday', 'weather', 'month', 'dayofweek', 'hour']

In [15]:
X = pd.get_dummies(X, columns=categorical_features_to_encode, drop_first=True)

In [16]:
# 중복 데이터 확인 및 제거
display(bike_df[bike_df.duplicated(keep=False)])
bike_df = bike_df.drop_duplicates()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,dayofweek,hour
5020,4,0,0,1,9.84,12.88,65,7.0015,1,10,11,2011,12,5,6
5188,4,0,0,1,9.84,12.88,65,7.0015,1,10,11,2011,12,5,6


In [17]:
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,dayofweek,hour
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,5,0
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,5,2
3,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,5,3
4,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,5,4


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)


In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

# 예측
y_pred = lr.predict(X_test)
print(y_pred[:3])

[-52.61062142 609.36944199  63.85181892]


In [53]:
mse = mean_squared_error(y_test, y_pred)
rmse= np.sqrt(mse)
mse, rmse

(9960.556495213596, np.float64(99.8025876178248))

In [54]:
r2_score(y_test, y_pred)

0.6880593617510383

In [55]:
#교차검증
r2_scores= cross_val_score(lr, X, y, scoring='r2', cv=5)
r2_scores, np.mean(r2_scores)

(array([0.01796541, 0.68464247, 0.56674454, 0.64808789, 0.63599471]),
 np.float64(0.510687001858995))

# 회귀

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)

In [57]:
rf = RandomForestRegressor(random_state=100, max_depth=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred[:3]

array([131.47836329, 648.78615112, 130.55214252])

In [58]:
mse = mean_squared_error(y_test, y_pred)
rmse= np.sqrt(mse)
mse, rmse

(20549.410226474767, np.float64(143.35065478216262))

In [59]:
r2_score(y_test, y_pred)

0.35644196739744327

In [60]:
r2_scores= cross_val_score(lr, X, y, scoring='r2', cv=5)
r2_scores, np.mean(r2_scores)

(array([0.01796541, 0.68464247, 0.56674454, 0.64808789, 0.63599471]),
 np.float64(0.510687001858995))

# 랜덤 포레스트

In [25]:
evals = [(X_test, y_test)]
xgb = XGBRegressor(n_estimators=400, learning_rate=0.1, max_depth=3, use_label_encoder=False)
xgb.fit(X_train, y_train,  eval_set=evals, verbose=True)
xgb_pred = xgb.predict(X_test)

[0]	validation_0-rmse:173.69849
[1]	validation_0-rmse:168.90888
[2]	validation_0-rmse:164.97530
[3]	validation_0-rmse:161.29738
[4]	validation_0-rmse:157.10349
[5]	validation_0-rmse:154.12413
[6]	validation_0-rmse:150.41231
[7]	validation_0-rmse:147.93784
[8]	validation_0-rmse:144.58433
[9]	validation_0-rmse:141.89182
[10]	validation_0-rmse:139.70914
[11]	validation_0-rmse:137.28878
[12]	validation_0-rmse:135.38865
[13]	validation_0-rmse:133.68776
[14]	validation_0-rmse:131.68030
[15]	validation_0-rmse:130.07039
[16]	validation_0-rmse:128.36336
[17]	validation_0-rmse:126.80331
[18]	validation_0-rmse:125.47059
[19]	validation_0-rmse:123.74619
[20]	validation_0-rmse:122.20471
[21]	validation_0-rmse:120.93981
[22]	validation_0-rmse:119.69388
[23]	validation_0-rmse:118.30367
[24]	validation_0-rmse:117.24213
[25]	validation_0-rmse:116.21873
[26]	validation_0-rmse:115.05361
[27]	validation_0-rmse:114.23029
[28]	validation_0-rmse:112.84168
[29]	validation_0-rmse:112.00638
[30]	validation_0-rm

c:\Users\tmd\miniconda3\envs\hipython\Lib\site-packages\xgboost\training.py:183: UserWarning: [05:09:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[64]	validation_0-rmse:92.88323
[65]	validation_0-rmse:92.65960
[66]	validation_0-rmse:92.40771
[67]	validation_0-rmse:91.07729
[68]	validation_0-rmse:90.69488
[69]	validation_0-rmse:90.37874
[70]	validation_0-rmse:89.99121
[71]	validation_0-rmse:89.15633
[72]	validation_0-rmse:88.93914
[73]	validation_0-rmse:88.67127
[74]	validation_0-rmse:88.50563
[75]	validation_0-rmse:88.12880
[76]	validation_0-rmse:87.54140
[77]	validation_0-rmse:86.52998
[78]	validation_0-rmse:86.32667
[79]	validation_0-rmse:86.14142
[80]	validation_0-rmse:85.83056
[81]	validation_0-rmse:85.32041
[82]	validation_0-rmse:84.82255
[83]	validation_0-rmse:84.63330
[84]	validation_0-rmse:84.39184
[85]	validation_0-rmse:84.16611
[86]	validation_0-rmse:83.99626
[87]	validation_0-rmse:83.87752
[88]	validation_0-rmse:83.42738
[89]	validation_0-rmse:82.87270
[90]	validation_0-rmse:82.69457
[91]	validation_0-rmse:82.51693
[92]	validation_0-rmse:81.85011
[93]	validation_0-rmse:81.48638
[94]	validation_0-rmse:81.16178
[95]	val

In [67]:
xgb_pred[:3]

array([  4.246787, 587.29956 , 151.30319 ], dtype=float32)

In [68]:
mse = mean_squared_error(y_test, xgb_pred)
rmse= np.sqrt(mse)
mse, rmse

(3001.461181640625, np.float64(54.78559282914282))

In [70]:
r2_score(y_test, xgb_pred)

0.9060014486312866

# XGBOOST

In [19]:
results = []

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=156)

In [21]:
model_poly = Pipeline([
  ('poly', PolynomialFeatures(degree=2, include_bias=False)),
  ('linear', LinearRegression())]
)
model_poly.fit(X_train, y_train)

,steps,"[('poly', ...), ('linear', ...)]"
,transform_input,None
,memory,None
,verbose,False
,degree,2
,interaction_only,False
,include_bias,False
,order,'C'
,fit_intercept,True
,copy_X,True
,tol,1e-06


In [22]:
pred_poly = model_poly.predict(X_test)
mean_squared_error(y_test, pred_poly)

2691.2583376982307

In [23]:
r2_score(y_test, pred_poly)

0.916376583776722

In [26]:
for degree in range(1,3):
  model_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
    ('linear', LinearRegression())]
  )
  model_poly.fit(X_train, y_train)
  pred_poly = model_poly.predict(X_test)
  mse = mean_squared_error(y_test, pred_poly)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, pred_poly)
  
  results.append({ 'degree':degree,
                  'MSE':mse,
                  'RMSE':rmse,
                  'R2': r2})

pd.DataFrame(results)

,degree,MSE,RMSE,R2
0,1,10234.100044,101.163729,0.682004
1,2,2691.258338,51.877339,0.916377
2,3,44313.128667,210.506838,-0.376908
3,1,10234.100044,101.163729,0.682004
4,2,2691.258338,51.877339,0.916377


In [27]:
r2_score(y_test, pred_poly)

0.916376583776722

In [29]:
results = []
best_r2=-np.inf
best_model = None
best_pred = None

for degree in range(1,3):
  model_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
    ('linear', LinearRegression())]
  )
  model_poly.fit(X_train, y_train)
  pred_poly = model_poly.predict(X_test)
  mse = mean_squared_error(y_test, pred_poly)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, pred_poly)
  
  results.append({ 'degree':degree,
                  'MSE':mse,
                  'RMSE':rmse,
                  'R2': r2})
  if r2> best_r2:
    best_r2 = r2
    best_model = model_poly
    best_pred = pred_poly
    
pd.DataFrame(results)

,degree,MSE,RMSE,R2
0,1,10234.100044,101.163729,0.682004
1,2,2691.258338,51.877339,0.916377


In [30]:
best_model.named_steps['poly'].degree

2

# 다항연산

In [31]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [32]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)

mse = mean_squared_error(y_test, pred_ridge)
r2  = r2_score(y_test, pred_ridge)
mse, r2

(10225.498661232248, 0.682270884715536)

In [35]:
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
ridge_cv = RidgeCV(alphas=alphas, cv=5)

In [36]:
ridge_cv.fit(X_train, y_train)
ridge_preds = ridge_cv.predict(X_test)

In [37]:
ridge_mse = mean_squared_error(y_test, ridge_preds)
ridge_r2 = r2_score(y_test, ridge_preds)
print(f'ridge cv mse: {ridge_mse:4f}, r2 : {ridge_r2:4f}')

ridge cv mse: 10234.086545, r2 : 0.682004


# 릿지

In [38]:
lasso = Lasso(alpha=0.1)  # alpha 값 작으면 규제 약해짐
lasso.fit(X_train, y_train)
pred_lasso = lasso.predict(X_test)

print("\n[라쏘 회귀]")
print("MSE:", mean_squared_error(y_test, pred_lasso))
print("R2:", r2_score(y_test, pred_lasso))


[라쏘 회귀]
MSE: 10252.020996819621
R2: 0.681446776425013


In [39]:
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
lasso_preds = lasso_cv.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_preds)
lasso_r2 = r2_score(y_test, lasso_preds)

In [40]:
print(f"[개선 Lasso] 최적 alpha: {lasso_cv.alpha_}")
print(f"MSE: {lasso_mse:.3f}, R2: {lasso_r2:.3f}")

[개선 Lasso] 최적 alpha: 0.001
MSE: 10233.608, R2: 0.682


# 라쏘

In [42]:
enet = ElasticNet(alpha=0.1, l1_ratio=0.5)
enet.fit(X_train, y_train)

,alpha,0.1
,l1_ratio,0.5
,fit_intercept,True
,precompute,False
,max_iter,1000
,copy_X,True
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [43]:
enet_pred = enet.predict(X_test)
print("\n[엘라스틱넷 회귀]")
print("MSE:", mean_squared_error(y_test, enet_pred))
print("R2:", r2_score(y_test, enet_pred))


[엘라스틱넷 회귀]
MSE: 14027.158320074444
R2: 0.564144815754611


# 엘라스틱넷 회귀

In [44]:
results = pd.DataFrame({
  '모델' : ['다항회귀', '릿지회귀', '라쏘회귀', '엘라스틱넷회귀'],
  'MSE': [ mean_squared_error(y_test, pred_poly),
          mean_squared_error(y_test, pred_ridge),
          mean_squared_error(y_test, pred_lasso),
          mean_squared_error(y_test, enet_pred),
        ],
  'R2': [r2_score(y_test, best_pred),
          r2_score(y_test, pred_ridge),
          r2_score(y_test, pred_lasso),
          r2_score(y_test, enet_pred),
  ]
})
results

,모델,MSE,R2
0,다항회귀,2691.258338,0.916377
1,릿지회귀,10225.498661,0.682271
2,라쏘회귀,10252.020997,0.681447
3,엘라스틱넷회귀,14027.158320,0.564145
